# Pound Drever Hall

## 0.0 Introduction

Pound Drever Hall is a technique to extract frequency offset information from a laser beam. This notebook is designed to give a nice visual introduction to PDH before diving deeper into some of the key math. 

## 1.0 Sinusoid Modulation

The first thing we will do is create a time-domain sinusoid, this will represent our laser beam. We then modulate this sinusoid via an electro-optic modulator (EOM). The EOM will periodically phase-shift the signal at some given modulation frequency. Both the modulated and unmodulated carrier waves are visible below. 

In [89]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# Generate time axis vectors
t1 = np.linspace(0, 1, 1000)
t2 = np.linspace(0, 10, 10000)



freq = FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1)
mod_freq = FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1)
beta = FloatSlider(value=0.1, min=0.01, max=5.0, step=0.01)
phase_shift = FloatSlider(value=180, min=0, max=360, step=1)


def sinusoid(freq, t, phase_shift_degrees=0):
    omega = 2 * np.pi * freq
    y = np.cos(omega * t + np.pi * phase_shift_degrees / 180) 
    return y

def modulated_sinusoid(freq, mod_freq, beta, t):
    omega = 2 * np.pi * freq
    y = np.cos(omega * t + beta * np.cos(2 * np.pi * mod_freq * t))
    return y

def get_sinusoid_spectrum(signal, t):
    N = len(t)
    dt = t[1] - t[0]
    Y = np.fft.fftshift(np.fft.fft(signal))
    f = np.fft.fftshift(np.fft.fftfreq(N, d=dt))
    Y_mag = np.abs(Y) / np.max(np.abs(Y))
    return f, Y_mag

def reflected_sinusoid(freq, mod_freq, beta, phase_shift, t):
    y = modulated_sinusoid(freq=freq, mod_freq=mod_freq, beta=beta, t=t) + sinusoid(freq=freq, phase_shift_degrees=phase_shift, t=t)
    return y

def plot_sinusoid(freq):
    T = t1
    y = sinusoid(freq=freq, t=T)
    plt.figure(figsize=(8,4))
    plt.plot(T, y, label=fr'$\cos(2\pi f t),\ f={freq:.2f}$ Hz')
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.title("Unmodulated Carrier")
    plt.grid(True)
    plt.legend()
    plt.show()

def plot_modulated_sinusoid(freq, mod_freq, beta):
    T = t1
    y = modulated_sinusoid(freq=freq, mod_freq=mod_freq, beta=beta, t=T)
    plt.figure(figsize=(8,4))
    plt.plot(T, y, label=fr'$\cos(2\pi f t),\ f={freq:.2f}$ Hz')
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.title("Modulated Carrier")
    plt.grid(True)
    plt.legend()
    plt.show()

def plot_spectrum_modulated_sinusoid(freq, mod_freq, beta):
    T = t2
    y = modulated_sinusoid(freq=freq, mod_freq=mod_freq, beta=beta, t=T)
    f, Y_mag = get_sinusoid_spectrum(signal=y, t=T)
    plt.figure(figsize=(10,5))
    plt.plot(f, Y_mag)
    plt.title("Frequency Spectrum of Phase-Modulated Signal")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Normalized Magnitude")
    plt.xlim(freq-5*mod_freq, freq+5*mod_freq)
    plt.grid(True)
    plt.show()

def plot_reflected_sinusoid(freq, mod_freq, beta, phase_shift):
    T=t1
    y = reflected_sinusoid(freq=freq, mod_freq=mod_freq, beta=beta, phase_shift=phase_shift, t=T)
    plt.figure(figsize=(8,4))
    plt.plot(T, y)
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.title("Reflected Field")
    plt.grid(True)
    plt.legend()
    plt.show()


In [90]:

interact(plot_sinusoid, freq=freq)
interact(plot_modulated_sinusoid, freq=freq, mod_freq=mod_freq, beta=beta)

interactive(children=(FloatSlider(value=1.0, description='freq', max=10.0, min=0.1), Output()), _dom_classes=(…

interactive(children=(FloatSlider(value=1.0, description='freq', max=10.0, min=0.1), FloatSlider(value=1.0, de…

<function __main__.plot_modulated_sinusoid(freq, mod_freq, beta)>

If we look at the modulated signal we see something that seems similar to multiple sinusoidal signals with different frequencies superimposing themselves on each other. This can be proven with the Jacobi-Anger identity but for now we'll just grab the frequency spectrum and see what happens. 

In [91]:


interact(plot_spectrum_modulated_sinusoid, 
         freq=freq, 
         mod_freq=mod_freq, 
         beta=beta)



interactive(children=(FloatSlider(value=1.0, description='freq', max=10.0, min=0.1), FloatSlider(value=1.0, de…

<function __main__.plot_spectrum_modulated_sinusoid(freq, mod_freq, beta)>

We can see that for modulation frequencies less than the carrier frequency, and for comparatively small values of beta, we generally observe two prominent 'sidebands' equidistant on each side of the dominant band (which as you'd expect is the carrier frequency). For now we can assume that a properly designed PDH system will ensure that the modulation frequency is much lower than the carrier frequency and that our beta is fairly small.

## 2.0 Cavity interference
We can make the assumption that our modulating signal is very resistant to frequency drift. If we could find a way to translate frequency offset into a phase offset, we could then get a beat signal by superimposing the two fields. We could then demodulate that beat signal with respect to our modulating tone to get an error signal. If the beat signal is odd-symmetric around our modulating tone, demodulating it with respect to that tone will allow us to obtain the magnitude and sign of our frequency offset. In other words, we should:

1. Impart a phase-shift on a signal with respect to frequency drift that is odd-symmetric
2. Make the phase shift as sharp as possible so we can detect it easily
3. Center it on our target carrier frequency that we are trying to correct to

We accomplish the three objectives above through either a Fabri-Perot cavity or a ring resonator. For what we care about they are functionally the same. We rotate the carrier with respect to frequency offset and demodulate the reflected field with respect to the modulation frequency.

For the code below, we assume a perfect cavity respose i.e. only the carrier component of our incident field is reflected. This is done to simplify the code. 

In [92]:
interact(plot_reflected_sinusoid, freq=freq, mod_freq=mod_freq, beta=beta, phase_shift=phase_shift)

interactive(children=(FloatSlider(value=1.0, description='freq', max=10.0, min=0.1), FloatSlider(value=1.0, de…

<function __main__.plot_reflected_sinusoid(freq, mod_freq, beta, phase_shift)>

. 